In [1]:
import datetime as dt

import bokeh as bk
import bokeh.models
import bokeh.plotting
import numpy as np
import pandas as pd
import pyperclip

In [2]:
bk.io.output_notebook()

Loading BokehJS ...

The sunrise equation:

$$
\cos H = -\tan \lambda \tan \delta
$$

Solving for the hour angle and substituting the declination of the mean sun, where $\alpha$ is the angle of the Sun from the vernal equinox and $\phi$ is the obliquity of the ecliptic:

$$
H = \arccos \left( -\tan \lambda \tan \left( \phi \sin \alpha \right) \right)
$$

The general derivative of the hour angle is

$$
\frac{dH}{d\alpha} = \frac{\phi \cos \alpha \tan \lambda \sec^2 (\phi \sin \alpha)}{\sqrt{1 - \tan^2 \lambda \tan^2 (\phi \sin \alpha)}}
$$

In [3]:
obliquity = np.pi * 23.45 / 180

In [4]:
year = 2023

In [5]:
days = pd.date_range(dt.datetime(year, 1, 1), dt.datetime(year + 1, 1, 1), periods=12*365).tolist()[:-1]

In [6]:
def daytime_date(dates, lat):
    alpha = [2 * np.pi * (elem - dt.datetime(year, 3, 20)).total_seconds() / (365 * 86400) for elem in dates]
    return 2 * 12 / np.pi * np.arccos(-np.tan(lat) * np.tan(obliquity * np.sin(alpha)))

In [7]:
def delta_day_date(dates, lat):
    alpha = [2 * np.pi * (elem - dt.datetime(year, 3, 20)).total_seconds() / (365 * 86400) for elem in dates]
    return (
        2 / 365 * 24 * 60 * obliquity * np.cos(alpha) * np.tan(lat) / np.cos(obliquity * np.sin(alpha))**2 /
        np.sqrt(1 - np.tan(lat)**2 * np.tan(obliquity * np.sin(alpha))**2)
    )

In [8]:
def add_quarter_days(fig):    
    quarter_days = [
        dt.datetime(year, 3, 20), dt.datetime(year, 6, 21), dt.datetime(year, 9, 22), dt.datetime(year, 12, 21)
    ]
    for quarter_day in quarter_days:
        span = bk.models.Span(
            location=quarter_day, dimension='height', line_width=2, line_dash='dotted', line_color='gray'
        )
        fig.add_layout(span)

In [9]:
def format_plot(fig):
    fig.xaxis.formatter = bk.models.formatters.DatetimeTickFormatter(months='%b')
    fig.xaxis.major_label_text_font_size = '12pt'
    fig.xaxis.major_label_text_font = 'garamound'
    fig.yaxis.major_label_text_font_size = '11pt'
    fig.yaxis.axis_label_text_font_size = '14pt'
    fig.yaxis.axis_label_text_font = 'garamound'
    fig.yaxis.axis_label_text_font_style = 'normal'
    fig.toolbar.active_drag = None
    fig.background_fill_alpha = 0
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    fig.title.text_font_size = '14pt'
    fig.title.text_font = 'garamound'
    fig.title.text_font_style = 'normal'
    fig.title.align = 'center'
    fig.title.text_align = 'center'
    add_quarter_days(fig)

In [10]:
plot_size = 450

daylight_source = bk.models.ColumnDataSource(data={'x': days, 'daylight': daytime_date(days, 34 * np.pi / 180)})
twelve_hour = bk.models.Span(location=12, dimension='width', line_width=1, line_color='gray')
zero_axis = bk.models.Span(location=0, dimension='width', line_width=1, line_color='gray')

daylight_hover_tool = bk.models.HoverTool(
    tooltips=[('Date', '@x{%b %d}'), ('Daylight', '@daylight')], formatters={'@x': 'datetime'}
)
daylight_fig = bk.plotting.figure(
    width=plot_size,
    height=plot_size,
    x_range=(dt.datetime(year, 1, 1), dt.datetime(year, 12, 31, 23, 59)),
    y_range=(0, 24),
    tools=['ywheel_zoom', 'ypan', daylight_hover_tool, 'reset'],
    toolbar_location='above',
    x_axis_type='datetime',
    margin=(0, 25),
)
daylight_fig.line('x', 'daylight', source=daylight_source, line_width=3)
daylight_fig.yaxis.axis_label = 'Hours of daylight'
format_plot(daylight_fig)
daylight_fig.add_layout(twelve_hour)

delta_source = bk.models.ColumnDataSource(data={'x': days, 'delta': delta_day_date(days, 34 * np.pi / 180)})
delta_hover_tool = bk.models.HoverTool(
    tooltips=[('Date', '@x{%b %d}'), ('Extra minutes', '@delta')], formatters={'@x': 'datetime'}
)
delta_fig = bk.plotting.figure(
    width=plot_size,
    height=plot_size,
    x_range=(dt.datetime(year, 1, 1), dt.datetime(year, 12, 31, 23, 59)),
    y_range=(-10, 10),
    tools=['ywheel_zoom', 'ypan', delta_hover_tool, 'reset'],
    toolbar_location='above',
    x_axis_type='datetime',
    margin=(0, 25),
)
delta_fig.xaxis.formatter = bk.models.formatters.DatetimeTickFormatter(months='%b')
delta_fig.yaxis.axis_label = 'Additional minutes of daylight'
delta_fig.line('x', 'delta', source=delta_source, line_width=3)
delta_fig.add_layout(zero_axis)
format_plot(delta_fig)

callback = bk.models.CustomJS(
    args={'daylight_source': daylight_source, 'delta_source': delta_source},
    code=f"""
    const lat = Math.PI * cb_obj.value / 180
    const equinox = new Date(Date.UTC({year}, 2, 20))
    var x = daylight_source.data.x
    var alpha = Array.from(x, (x) => 2 * Math.PI * (x - equinox.getTime()) / 1000 / 365 / 86400)
    const obliquity = Math.PI * 23.45 / 180
    
    const daylight = Array.from(alpha, (alpha) => 2 * 12 / Math.PI * 
        Math.acos(-Math.tan(lat) * Math.tan(obliquity * Math.sin(alpha))))
    daylight_source.data = {{ x, daylight }}

    x = delta_source.data.x
    const delta = Array.from(alpha, (alpha) => 2 / 365 * 24 * 60 * obliquity * Math.cos(alpha) * Math.tan(lat) /
        Math.pow(Math.cos(obliquity * Math.sin(alpha)), 2) /
        Math.sqrt(1 - Math.pow(Math.tan(lat) * Math.tan(obliquity * Math.sin(alpha)), 2)))
    delta_source.data = {{ x, delta }}
""")

slider = bk.models.Slider(
    start=0,
    end=90,
    value=34,
    step=0.01,
    title='Latitude',
    width=800,
    margin=(25, 100),
)
slider.js_on_change('value', callback)

In [11]:
theme = bk.themes.Theme(
    json={
        'attrs': {
            'Plot': {
                'background_fill_color': 'rgba(0, 0, 0, 0)',
                'border_fill_color': 'rgba(0, 0, 0, 0)',
            }
        }
    }
)

In [12]:
bk_app = bk.layouts.column(
    bk.layouts.row(daylight_fig, delta_fig, background=None), slider, background=None
)

In [13]:
bk.plotting.show(bk_app)

In [14]:
script, div = bk.embed.components(bk_app, theme=theme)

## Correction for atmospheric refraction

In [15]:
def extra_daylight_atm(dates, lat):
    alpha = [2 * np.pi * (elem - dt.datetime(year, 3, 20)).total_seconds() / (365 * 86400) for elem in dates]
    dec = obliquity * np.sin(alpha)
    H_atm = np.arccos(
        -np.tan(lat) * np.tan(dec) + np.sin(50/60 * np.pi/180) / (np.cos(lat) * np.cos(dec))
    )
    H_no_atm = np.arccos(-np.tan(lat) * np.tan(dec))
    return -2 * 60 * 12 / np.pi * (H_atm - H_no_atm)

In [16]:
atm_source = bk.models.ColumnDataSource(data={'x': days, 'atm_delta': extra_daylight_atm(days, 34 * np.pi / 180)})
atm_hover_tool = bk.models.HoverTool(
    tooltips=[('Date', '@x{%b %d}'), ('Extra minutes', '@atm_delta')], formatters={'@x': 'datetime'}
)
atm_fig = bk.plotting.figure(
    width=plot_size,
    height=plot_size,
    x_range=(dt.datetime(year, 1, 1), dt.datetime(year, 12, 31, 23, 59)),
    y_range=(0, 30),
    tools=['ywheel_zoom', 'ypan', atm_hover_tool, 'reset'],
    toolbar_location='right',
    x_axis_type='datetime',
    title='Extra daylight from atmospheric\nrefraction and solar limb',
)
format_plot(atm_fig)
atm_fig.yaxis.axis_label = 'Minutes'
atm_fig.line('x', 'atm_delta', source=atm_source, line_width=3)
atm_fig.add_layout(zero_axis)

callback = bk.models.CustomJS(
    args={'atm_source': atm_source},
    code=f"""
    const lat = Math.PI * cb_obj.value / 180
    const equinox = new Date(Date.UTC({year}, 2, 20))
    var x = atm_source.data.x
    var alpha = Array.from(x, (x) => 2 * Math.PI * (x - equinox.getTime()) / 1000 / 365 / 86400)
    const obliquity = Math.PI * 23.45 / 180
    
    const atm_delta = Array.from(alpha, (alpha) => -2 * 60 * 12 / Math.PI * (
        Math.acos(-Math.tan(lat) * Math.tan(obliquity * Math.sin(alpha)) +
            Math.sin(5/6*Math.PI/180) / (Math.cos(lat) * Math.cos(obliquity * Math.sin(alpha))))
        - Math.acos(-Math.tan(lat) * Math.tan(obliquity * Math.sin(alpha)))
    ))
    atm_source.data = {{ x, atm_delta }}
""")

atm_slider = bk.models.Slider(
    start=0,
    end=90,
    value=34,
    step=0.01,
    title='Latitude',
    width=400,
    margin=(25, 35),
)
atm_slider.js_on_change('value', callback)

In [17]:
bk_app = bk.layouts.column(atm_fig, atm_slider, background=None)

In [18]:
bk.plotting.show(bk_app)

In [19]:
script, div = bk.embed.components(bk_app, theme=theme)